In [29]:
import tarfile
import scipy.io
import numpy as np
import os
import cv2 as cv
import shutil
import random
from console_progressbar import ProgressBar




In [30]:
def ensure_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [31]:
def process_train_data():
    print("Processing train data...")
    cars_annos = scipy.io.loadmat('devkit//cars_train_annos.mat')
    annotations = cars_annos['annotations']
    annotations = np.transpose(annotations)

    fnames = []
    class_ids = []
    bboxes = []
    labels = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        class_id = annotation[0][4][0][0]
        labels.append('%04d' % (class_id,))
        fname = annotation[0][5][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        class_ids.append(class_id)
        fnames.append(fname)

    labels_count = np.unique(class_ids).shape[0]
    print(np.unique(class_ids))
    print('The number of different cars is %d' % labels_count)

    save_train_data(fnames, labels, bboxes)


In [32]:
def save_train_data(fnames, labels, bboxes):
    src_folder ='cars_train' #Change this to your cars_train path
    num_samples = len(fnames)

    train_split = 0.8
    num_train = int(round(num_samples * train_split))
    train_indexes = random.sample(range(num_samples), num_train)

    pb = ProgressBar(total=100, prefix='Save train data', suffix='', decimals=3, length=50, fill='=')

    for i in range(num_samples):
        fname = fnames[i]
        label = labels[i]
        (x1, y1, x2, y2) = bboxes[i]

        src_path = os.path.join(src_folder, fname)
        
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)
        # print("{} -> {}".format(fname, label))
        pb.print_progress_bar((i + 1) * 100 / num_samples)

        if i in train_indexes:
            dst_folder = 'data//train//'
        else:
            dst_folder = 'data//valid//'

        dst_path = os.path.join(dst_folder, label)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        dst_path = os.path.join(dst_path, fname)

        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=src_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)

In [33]:
def process_test_data():
    print("Processing test data...")
    cars_annos = scipy.io.loadmat('devkit//cars_test_annos_withlabels.mat') #Change this to your devkit path
    annotations = cars_annos['annotations']
    annotations = np.transpose(annotations)

    fnames = []
    class_ids = []
    bboxes = []
    labels = []

    for annotation in annotations:
        bbox_x1 = annotation[0][0][0][0]
        bbox_y1 = annotation[0][1][0][0]
        bbox_x2 = annotation[0][2][0][0]
        bbox_y2 = annotation[0][3][0][0]
        class_id = annotation[0][4][0][0]
        labels.append('%04d' % (class_id,))
        fname = annotation[0][5][0]
        bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
        class_ids.append(class_id)
        fnames.append(fname)

    labels_count = np.unique(class_ids).shape[0]
    print(np.unique(class_ids))
    print('The number of different cars is %d' % labels_count)

    save_test_data(fnames, labels, bboxes)


In [34]:
def save_test_data(fnames, labels, bboxes):
    src_folder ='cars_test' #Change this to your cars_test path
    dst_folder = 'data//test//'
    num_samples = len(fnames)
    pb = ProgressBar(total=100, prefix='Save test data', suffix='', decimals=3, length=50, fill='=')

    for i in range(num_samples):
        fname = fnames[i]
        label = labels[i]
        (x1, y1, x2, y2) = bboxes[i]

        src_path = os.path.join(src_folder, fname)
        src_image = cv.imread(src_path)
        height, width = src_image.shape[:2]
        # margins of 16 pixels
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)
        # print("{} -> {}".format(fname, label))
        pb.print_progress_bar((i + 1) * 100 / num_samples)

        dst_path = os.path.join(dst_folder, label)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        dst_path = os.path.join(dst_path, fname)

        crop_image = src_image[y1:y2, x1:x2]
        dst_img = cv.resize(src=src_image, dsize=(img_height, img_width))
        cv.imwrite(dst_path, dst_img)

## Process Train, Valid, and Test Data

In [35]:
if __name__ == '__main__':
    # parameters
    img_width, img_height = 224, 224

    print('Extracting cars_train.tgz...')
    if not os.path.exists('cars_train'):
        with tarfile.open('cars_train.tgz', "r:gz") as tar: #Change to your tgz path
            tar.extractall()
    print('Extracting cars_test.tgz...')
    if not os.path.exists('cars_test'):
        with tarfile.open('cars_test.gz', "r:gz") as tar: #Change to your tgz path
            tar.extractall()
    print('Extracting car_devkit.tgz...')
    if not os.path.exists('devkit'):
        with tarfile.open('car_devkit.tgz', "r:gz") as tar: #Change to your tgz path
            tar.extractall()

    cars_meta = scipy.io.loadmat('devkit/cars_meta')
    class_names = cars_meta['class_names']  # shape=(1, 196)
    class_names = np.transpose(class_names)
    print('class_names.shape: ' + str(class_names.shape))
    print('Sample class_name: [{}]'.format(class_names[8][0][0]))

    ensure_folder('data/train')
    ensure_folder('data/valid')
    ensure_folder('data/test')


    process_train_data()
    process_test_data()

Extracting cars_train.tgz...
Extracting cars_test.tgz...
Extracting car_devkit.tgz...
class_names.shape: (196, 1)
Sample class_name: [Aston Martin V8 Vantage Coupe 2012]
Processing train data...
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196]
The numbe

## Rename Files (For Aesthethic Purposes)

In [36]:
cars_meta = scipy.io.loadmat('devkit/cars_meta')
class_names = cars_meta['class_names']  # shape=(1, 196)
class_names = np.transpose(class_names)
print('class_names.shape: ' + str(class_names.shape))
print('Sample class_name: [{}]'.format(class_names[8][0][0]))
import re
new_classnames = []
regexp = re.compile('/')
for i in range(class_names.shape[0]):
    if regexp.search(class_names[i][0][0]):
        cl_temp = re.sub('/','-',class_names[i][0][0])
        print("Changed",class_names[i][0][0],"to",cl_temp)
        new_classnames.append(cl_temp)
    else:
        new_classnames.append(class_names[i][0][0])
        
datadir = os.listdir('data')
for i in datadir:
    filename = os.listdir(os.path.join('data', i))
    for j in range(len(filename)):
        os.rename(os.path.join('data',i,filename[j]),os.path.join('data',i,str("{:05d}".format(j+1))+" "+str(new_classnames[j])))

class_names.shape: (196, 1)
Sample class_name: [Aston Martin V8 Vantage Coupe 2012]
Changed Ram C/V Cargo Van Minivan 2012 to Ram C-V Cargo Van Minivan 2012


## Augment Data with Sharpen

Augmenting the training data with sharpen will allow the model to learn more detailed features, and also give the model more training data

In [2]:
import os
from PIL import Image
from PIL import ImageFilter

dir_path = 'data/train'
filelist = os.listdir(dir_path)
for i in range(len(filelist)):
    class_path = os.listdir(os.path.join(dir_path,filelist[i]))
    print(filelist[i])
    for j in range(len(class_path)):
        img = os.path.join(dir_path,filelist[i],class_path[j])
        im=Image.open(img)
        im=im.convert("RGB")
        im_sharp=im.filter(ImageFilter.SHARPEN)
        im_sharp.save(os.path.join(dir_path,filelist[i],'sh_'+class_path[j]))
        print(class_path[j], end = "\r")

00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM General Hummer SUV 2000
00001 AM G

00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00010 Aston Martin Virage Convertible 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
00011 Aston Martin Virage Coupe 2012
000

00019 Audi TT Hatchback 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00020 Audi S6 Sedan 2011
00021 Audi S5 Convertible 2012
00021 Audi S5 Convertible 2012
00021 Aud

00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00029 BMW 3 Series Sedan 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 3 Series Wagon 2012
00030 BMW 

00038 BMW Z4 Convertible 2012
00038 BMW Z4 Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental Supersports Conv. Convertible 2012
00039 Bentley Continental

...

00161 Mercedes-Benz 300-Class Convertible 1993
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Benz C-Class Sedan 2012
00162 Mercedes-Be

00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00168 Nissan Leaf Hatchback 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV Passenger Van 2012
00169 Nissan NV

00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Ghost Sedan 2012
00176 Rolls-Royce Gh

00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00185 Tesla Model S Sedan 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequoia SUV 2012
00186 Toyota Sequ

00192 Volkswagen Beetle Hatchback 2012
00192 Volkswagen Beetle Hatchback 2012
00192 Volkswagen Beetle Hatchback 2012
00192 Volkswagen Beetle Hatchback 2012
00192 Volkswagen Beetle Hatchback 2012
00192 Volkswagen Beetle Hatchback 2012
00192 Volkswagen Beetle Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C30 Hatchback 2012
00193 Volvo C3